# Agriculture Supply Chain ETL Process

In [2]:
!pip install sqlalchemy

   ---------------------------------------- 0.0/2.1 MB ? eta -:--:--
   ---------------------------------------- 0.0/2.1 MB ? eta -:--:--
   --------- ------------------------------ 0.5/2.1 MB 2.1 MB/s eta 0:00:01
   -------------- ------------------------- 0.8/2.1 MB 1.9 MB/s eta 0:00:01
   ------------------- -------------------- 1.0/2.1 MB 1.7 MB/s eta 0:00:01
   ------------------------ --------------- 1.3/2.1 MB 1.3 MB/s eta 0:00:01
   ----------------------------- ---------- 1.6/2.1 MB 1.4 MB/s eta 0:00:01
   ---------------------------------------- 2.1/2.1 MB 1.6 MB/s eta 0:00:00

   ---------------------------------------- 0/2 [greenlet]
   ---------------------------------------- 0/2 [greenlet]
   ---------------------------------------- 0/2 [greenlet]
   ---------------------------------------- 0/2 [greenlet]
   -------------------- ------------------- 1/2 [sqlalchemy]
   -------------------- ------------------- 1/2 [sqlalchemy]
   -------------------- ------------------- 1/2

In [1]:
!pip install pandas
!pip install pyodbc

In [16]:
# Import required libraries
import pandas as pd
import pyodbc
from datetime import datetime
import logging
import numpy as np
from sqlalchemy import create_engine

# Configure logging
logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(levelname)s - %(message)s'
)
logger = logging.getLogger(__name__)

# Extract

Mengekstrak data mentah dari sumber untuk dipersiapkan dalam proses transformasi dan pemuatan (ETL) ke dalam database SQL Server. Data mentah yang disiapkan berupa data XML, Excel, dan data gabungan transaksi

In [42]:
import pandas as pd
import xml.etree.ElementTree as ET
from pathlib import Path
import logging

logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(levelname)s - %(message)s'
)
logger = logging.getLogger(__name__)

def bacaCSVFile(file_path):
    try:
        df = pd.read_csv(file_path)
        logger.info(f"Successfully read CSV file: {file_path}")
        return df
    except Exception as e:
        logger.error(f"Error reading CSV file {file_path}: {str(e)}")
        return None

def bacaExcelFile(file_path):
    try:
        df = pd.read_excel(file_path)
        logger.info(f"Successfully read Excel file: {file_path}")
        return df
    except Exception as e:
        logger.error(f"Error reading Excel file {file_path}: {str(e)}")
        return None

def bacaXMLFile(file_path):
    try:
        # Parse XML file
        tree = ET.parse(file_path)
        root = tree.getroot()
        
        # Convert XML to list of dictionaries
        data = []
        for child in root:
            row = {}
            for elem in child:
                row[elem.tag] = elem.text
            data.append(row)
        
        # Convert to DataFrame
        df = pd.DataFrame(data)
        logger.info(f"Successfully read XML file: {file_path}")
        return df
    except Exception as e:
        logger.error(f"Error reading XML file {file_path}: {str(e)}")
        return None

def read_all_raw_data():
    raw_data_dir = Path('raw_data')
    data_dict = {}
    
    # Check if directory exists
    if not raw_data_dir.exists():
        logger.error(f"Directory {raw_data_dir} does not exist!")
        return None
    
    # Read all files in the directory
    for file_path in raw_data_dir.glob('*'):
        if file_path.is_file():
            file_extension = file_path.suffix.lower()
            file_name = file_path.stem
            
            logger.info(f"Processing file: {file_name}")
            
            if file_extension == '.csv':
                df = bacaCSVFile(file_path)
            elif file_extension == '.xlsx':
                df = bacaExcelFile(file_path)
            elif file_extension == '.xml':
                df = bacaXMLFile(file_path)
            else:
                logger.warning(f"Unsupported file format: {file_extension} for file {file_name}")
                continue
            
            if df is not None:
                data_dict[file_name] = df
                # Display basic information about the dataset
                print(f"\nDataset Info for {file_name}:")
                print(f"Shape: {df.shape}")
                print("\nColumns:")
                print(df.columns.tolist())
                print("\nFirst 5 rows:")
                print(df.head())
                print("\n" + "="*50)
    
    return data_dict

if __name__ == "__main__":
    # Read all raw data
    raw_data = read_all_raw_data()
    
    if raw_data:
        print(f"\nSuccessfully read {len(raw_data)} files:")
        for file_name in raw_data.keys():
            print(f"- {file_name}")
    else:
        print("No data was read successfully.")

2025-05-27 21:36:23,238 - INFO - Processing file: data_ritel
2025-05-27 21:36:23,242 - INFO - Successfully read CSV file: raw_data\data_ritel.csv
2025-05-27 21:36:23,245 - INFO - Processing file: gudang_stok
2025-05-27 21:36:23,251 - INFO - Successfully read CSV file: raw_data\gudang_stok.csv
2025-05-27 21:36:23,254 - INFO - Processing file: produksi
2025-05-27 21:36:23,258 - INFO - Successfully read CSV file: raw_data\produksi.csv
2025-05-27 21:36:23,262 - INFO - Processing file: profil_petani
2025-05-27 21:36:23,272 - INFO - Successfully read XML file: raw_data\profil_petani.xml
2025-05-27 21:36:23,276 - INFO - Processing file: sukamarga_logistik
2025-05-27 21:36:23,432 - INFO - Successfully read Excel file: raw_data\sukamarga_logistik.xlsx
2025-05-27 21:36:23,435 - INFO - Processing file: transaksi_pertanian
2025-05-27 21:36:23,440 - INFO - Successfully read CSV file: raw_data\transaksi_pertanian.csv



Dataset Info for data_ritel:
Shape: (1000, 3)

Columns:
['retail_partner_id', 'name', 'type']

First 5 rows:
   retail_partner_id                                   name         type
0                  1  CV Mangunsong Wulandari (Persero) Tbk  Distributor
1                  2                    PT Nasyidah Pratiwi  Distributor
2                  3                 Perum Waluyo Anggraini  Distributor
3                  4                      CV Prastuti Tamba  Supermarket
4                  5                    CV Namaga Iswahyudi  Distributor


Dataset Info for gudang_stok:
Shape: (1000, 3)

Columns:
['warehouse_id', 'location', 'capacity_kg']

First 5 rows:
   warehouse_id     location  capacity_kg
0             1     Bengkulu         7774
1             2        Batam         8669
2             3  Tasikmalaya        10306
3             4    Mojokerto         7983
4             5      Mataram         5872


Dataset Info for produksi:
Shape: (1000, 4)

Columns:
['product_id', 'product_na

## Transform

Melakukan transformasi terhadap seluruh data mentah, termasuk pembersihan dan pencocokan struktur kolom, kemudian menyimpannya dalam format CSV.

---

### Transform XML to CSV

Mengubah data mentah XML kedalam format CSV dengan penyesuaian kolom

In [11]:
import pandas as pd
import xml.etree.ElementTree as ET

def xml_to_csv(xml_file, csv_file):
    # Parse XML file
    tree = ET.parse(xml_file)
    root = tree.getroot()
    
    # Convert XML to list of dictionaries
    data = []
    for child in root:
        row = {}
        for elem in child:
            row[elem.tag] = elem.text
        data.append(row)
    
    # Convert to DataFrame and save as CSV
    df = pd.DataFrame(data)
    df.to_csv(csv_file, index=False)
    print(f"Successfully converted {xml_file} to {csv_file}")
    print(f"Number of records: {len(df)}")
    print("\nFirst 5 rows of the data:")
    print(df.head())

if __name__ == "__main__":
    # Convert XML to CSV
    xml_file = "profil_petani.xml"
    csv_file = "warehouse/dataset/Dim_Farmer.csv"
    xml_to_csv(xml_file, csv_file)

Successfully converted profil_petani.xml to good/Dim_Farmer.csv
Number of records: 1000

First 5 rows of the data:
  farmer_id                     name      location farm_size_ha
0         1          Elvina Prasasta    Purwokerto         1.77
1         2         Cut Tantri Hakim      Sukabumi         3.11
2         3            Indah Gunawan  Tanjungbalai         4.49
3         4      Hani Santoso, S.E.I    Banda Aceh         1.94
4         5  Hj. Talia Widodo, S.Kom         Medan         1.98


### Transform XLSX to CSV

Mengubah data excel kedalam csv

In [12]:
import pandas as pd

# Read Excel file
df = pd.read_excel('sukamarga_logistik.xlsx')

# Display basic info
print("DataFrame Info:")
print(df.info())
print("\nFirst 5 rows:")
print(df.head())

# If you want to see the raw data as CSV format without saving
print("\nCSV format:")
print(df.to_csv(index=False))
df.to_csv('warehouse/dataset/Dim_Logistics.csv', index=False)

DataFrame Info:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 7 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   shipment_id       1000 non-null   int64  
 1   date              1000 non-null   int64  
 2   origin            1000 non-null   int64  
 3   destination       1000 non-null   int64  
 4   distance_km       1000 non-null   float64
 5   duration_minutes  1000 non-null   int64  
 6   delivery_status   1000 non-null   object 
dtypes: float64(1), int64(5), object(1)
memory usage: 54.8+ KB
None

First 5 rows:
   shipment_id   date  origin  destination  distance_km  duration_minutes  \
0            0  45814      23          289       147.69               524   
1            1  45745      16          160       407.50               385   
2            2  45680     620           97       389.17               184   
3            3  45733     698          425       217.07          

### Denormalized Transaction
Memisahkan transaksi dengan waktu dan membuat 2 dataset berbeda (Transaction & Time)

In [13]:
import pandas as pd

# Read the original dataset
df = pd.read_csv('transaksi_pertanian.csv')

# Create Dim_Time
dim_time = df[['time_id', 'date', 'month', 'kuarter', 'years']].drop_duplicates()
print("\nDim_Time Preview:")
print(dim_time.head())
print("\nDim_Time Shape:", dim_time.shape)

# Create Fact_Transactions
fact_transactions = df[['transaction_id', 'time_id', 'product_id', 'farmer_id', 
                       'warehouse_id', 'retail_partner_id', 'quantity_kg', 
                       'price_per_kg', 'shipping_cost', 'shipment_id']]
print("\nFact_Transactions Preview:")
print(fact_transactions.head())
print("\nFact_Transactions Shape:", fact_transactions.shape)

# If you want to see the data in CSV format without saving
print("\nDim_Time CSV format:")
print(dim_time.to_csv('warehouse/dataset/Dim_Time.csv',index=False))

print("\nFact_Transactions CSV format:")
print(fact_transactions.to_csv('warehouse/dataset/Fact_Transactions.csv',index=False))


Dim_Time Preview:
   time_id        date  month  kuarter  years
0      177  2025-01-16      1        1   2025
1      209  2025-06-30      6        2   2025
2      194  2025-07-30      7        3   2025
3      534  2025-03-13      3        1   2025
4      992  2025-07-17      7        3   2025

Dim_Time Shape: (91, 5)

Fact_Transactions Preview:
   transaction_id  time_id  product_id  farmer_id  warehouse_id  \
0               1      177          32        850           191   
1               2      209         245        544           660   
2               3      194         303        775           905   
3               4      534         817        610           394   
4               5      992         535         48           660   

   retail_partner_id  quantity_kg  price_per_kg  shipping_cost  shipment_id  
0                413        85.16       4168.38      105885.62          714  
1                778        78.99       3153.62      158966.98          578  
2              

In [14]:
stok = pd.read_csv("gudang_stok.csv")
produk = pd.read_csv("produksi.csv")
retail = pd.read_csv("data_ritel.csv")


stok.to_csv("warehouse/dataset/Dim_Warehouse.csv",index=False)
produk.to_csv("warehouse/dataset/Dim_Product.csv",index=False)
retail.to_csv("warehouse/dataset/Dim_Retail.csv",index=False)

# pd.read_csv("gudang_stok.csv")


### Load

Data hasil transformasi kemudian dimuat (load) ke dalam tabel-tabel pada SQL Server untuk membentuk skema data warehouse.

In [35]:
# SQL Server connection parameters
SERVER = '.'
DATABASE = 'agri'
USERNAME = 'MUEL\\diosa'
PASSWORD = ''

# Create connection string
# conn_str = f'mssql+pyodbc://{USERNAME}:{PASSWORD}@{SERVER}/{DATABASE}?driver=ODBC+Driver+17+for+SQL+Server'
conn_str = f'mssql+pyodbc://{SERVER}/{DATABASE}?driver=ODBC+Driver+17+for+SQL+Server&Trusted_Connection=yes'
engine = create_engine(conn_str)

In [41]:
from sqlalchemy import create_engine, text
import pyodbc
from urllib.parse import quote_plus

def loadETLSQLServer(csv_url: str, table_name: str):
    try:
        # Construct the BULK INSERT command
        bulk_insert_query = f"""
        BULK INSERT dbo.{table_name}
        FROM '{csv_url}'
        WITH (
            FORMAT = 'CSV',
            FIRSTROW = 2,           -- Skip header row
            FIELDTERMINATOR = ',',  -- Column separator
            ROWTERMINATOR = '\\n',  -- Row separator
            TABLOCK
        );
        """
        
        print(f"\nExecuting BULK INSERT for {table_name}")
        print(f"Source file: {csv_url}")
        
        # Execute the BULK INSERT command using engine
        with engine.connect() as connection:
            # Execute the bulk insert
            connection.execute(text(bulk_insert_query))
            connection.commit()
            
            # Verify the insertion
            result = connection.execute(text(f"SELECT COUNT(*) as total FROM {table_name}")).fetchone()
            total_rows = result[0]
            print(f"\nTotal records inserted: {total_rows}")
        
        print("\nBulk insert completed successfully!")
        return True
        
    except Exception as e:
        print(f"Error during bulk insert: {str(e)}")
        return False

# Example usage:
if __name__ == "__main__":
    # Example parameters
    csv_path = r"C:\Users\diosa\OneDrive\Desktop\program\Agrikultur-Supply-Chain-Kelompok-13\warehouse\dataset\Dim_Logistics.csv"
    table_name = "Dim_Logistics"
    
    # Call the function
    success = loadETLSQLServer(csv_path, table_name)
    
    if success:
        print("Data insertion completed successfully!")
    else:
        print("Data insertion failed!")


Executing BULK INSERT for Dim_Logistics
Source file: C:\Users\diosa\OneDrive\Desktop\program\Agrikultur-Supply-Chain-Kelompok-13\warehouse\dataset\Dim_Logistics.csv

Total records inserted: 1000

Bulk insert completed successfully!
Data insertion completed successfully!
